## 트러블슈팅: 실전 문제 진단 매트릭스


**학습 오차 분석**  
- **과적합(Overfitting) 진단**:  
  - 훈련 데이터 정확도는 높지만 검증 데이터 성능이 저하될 경우 발생  
  - 해결 방법: 드롭아웃 적용, 데이터 증강, L2 정규화 계수 조정  
  ```python
  model.add(Dropout(0.5))  # 50% 드롭아웃 적용 예시
  ```

- **과소적합(Underfitting) 식별**:  
  - 훈련/검증 정확도 모두 낮을 때 주로 발생  
  - 해결 전략: 모델 복잡도 증가(레이어 추가), 학습률 조정

**추세 반영 실패 진단**  
- 시계열 예측 모델에서 주로 발생하는 문제  
- **진단 지표**:  
  - MAPE(Mean Absolute Percentage Error) > 15%  
  - 예측 곡선과 실제 데이터 추세선 간 상관계수 < 0.7  
- **대응 방안**:  
  - 윈도우 크기 조정(과거 데이터 참조 기간 변경)  
  - LSTM 계층 추가를 통한 장기 의존성 학습 강화



### 하이퍼파라미터 최적화 기법

**주요 튜닝 대상 파라미터**  
| 파라미터 | 영향 범위 | 최적화 목표 |  
|---------|----------|-------------|  
| 학습률(Learning Rate) | 전역 최적점 수렴 속도/최적점(최소값) 지나침 | 0.001~0.1 범위 탐색 |  
| 배치 크기(Batch Size) | 메모리 사용량/학습 안정성 | 32~256 사이 권장 |  
| 에포크(Epochs) | 과소적합 방지 | Early Stopping 적용 |  

기타: Window Size, Dropout Ratio

**최적화 알고리즘 비교**  
| 방법 | 장점 | 단점 | 적용 시나리오 |  
|------|------|------|---------------|  
| 그리드 서치 | 체계적 탐색 | 계산 비용 높음 | 소규모 파라미터 공간 |  
| 랜덤 서치 | 빠른 수렴 | 결과 불확실성 | 넓은 탐색 영역 |  
| 베이지안 최적화 | 지능적 탐색 | 구현 복잡성 | 고비용 실험 환경 |  

---
```python 
# 그리드 서치 예제
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# 파라미터 그리드 정의
param_grid = {
    'C': [0.1, 1, 10],          # 규제 강도
    'kernel': ['linear', 'rbf'] # 커널 종류
}

# 모델 생성
model = SVC()

# 5-fold 교차검증 설정
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy'
)

# 학습 실행
grid_search.fit(X_train, y_train)

# 최적 파라미터 출력
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_:.4f}")

```
---
```python
# 랜덤 서치 예제
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# 파라미터 분포 정의
param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3)
}

# 모델 생성
model = XGBClassifier()

# 500회 무작위 샘플링
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=500,
    cv=3,
    scoring='neg_mean_squared_error'
)

# 학습 실행
random_search.fit(X_train, y_train)

# 결과 출력
print(f"Best parameters: {random_search.best_params_}")

```
---
```python
# 베이지안 최적화 예제
from skopt import BayesSearchCV
from skopt.space import Real, Integer

# 검색 공간 정의
search_spaces = {
    'learning_rate': Real(0.001, 0.1, prior='log-uniform'),
    'max_depth': Integer(3, 12),
    'subsample': Real(0.5, 1.0)
}

# 모델 생성
model = XGBRegressor()

# 베이지안 최적화 설정
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=search_spaces,
    n_iter=50,          # 평가 횟수
    cv=5,
    scoring='r2'
)

# 학습 실행
bayes_search.fit(X_train, y_train)

# 결과 분석
print(f"Best parameters: {bayes_search.best_params_}")

```


### 종합 전략
문제 진단 단계에서는 **학습 곡선 분석**과 **오차 분해 기법**을 결합하여 근본 원인을 규명해야 함.    
하이퍼파라미터 최적화 시에는 `BayesianOptimization` 패키지 등을 활용해 계산 효율성을 높일 수 있으며,    
실전 적용 시 모니터링 지표로 **변경 실패율**을 추가 추적해야 함.


#### 학습 곡선 분석
- 모델의 학습 진행 상황을 시각화한 그래프**로, 
- 주로 훈련 데이터와 검증 데이터의 손실(loss) 및 정확도(accuracy) 추이를 비교
- 이상적인 패턴은 두 데이터셋의 성능이 함께 개선되며 간격이 좁은 상태

##### 주요 진단 시나리오
| 현상 | 원인 | 해결 방안 |
|------|------|----------|
| **훈련/검증 오차 모두 높음** | 과소적합(Underfitting) | 모델 복잡도 증가, 학습률 조정 |
| **훈련 오차 낮음 + 검증 오차 높음** | 과적합(Overfitting) | 드롭아웃 적용, 데이터 증강 |
| **검증 오차가 갑작스럽게 상승** | 조기 종료(Early Stopping) 필요 | 검증 손실 모니터링 |

```python
# TensorFlow/Keras 학습 곡선 시각화 예제
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Learning Curve')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()
```
---
#### 오차 분해 기법
- 오차를 **편향(Bias), 분산(Variance), 노이즈(Noise)**로 분해하여 문제 원인을 정량화
- 모델이 과적합(overfitting)인지, 과소적합(underfitting)인지, 또는 데이터 자체의 노이즈가 큰지 진단하는 데 사용
$$
\text{Total Error} = \text{Bias}^2 + \text{Variance} + \text{Noise}
$$
- **Bias(편향)**: 모델이 데이터의 실제 패턴을 얼마나 잘 포착하는지  
- **Variance(분산)**: 데이터 샘플이 바뀔 때 예측이 얼마나 달라지는지  
- **Noise(노이즈)**: 데이터 자체의 불가피한 오차

##### 구현 예시
- 가장 대표적으로는 여러 번의 샘플링과 모델 학습을 반복하여 각 요소를 추정하는 방법이 있음

```python
import numpy as np
from sklearn.utils import resample
from sklearn.metrics import mean_squared_error

def bias_variance_decomposition(model, X, y, n_trials=100):
    predictions = np.zeros((n_trials, len(X)))
    for i in range(n_trials):
        X_resampled, y_resampled = resample(X, y)
        model.fit(X_resampled, y_resampled)
        predictions[i] = model.predict(X)
    # 평균 예측값
    mean_pred = np.mean(predictions, axis=0)
    # Bias^2
    bias_squared = np.mean((mean_pred - y) ** 2)
    # Variance
    variance = np.mean(np.var(predictions, axis=0))
    # Noise (데이터의 내재적 오차, 보통 측정 불가, 실제로는 무시하거나 추정)
    return bias_squared, variance

# 사용 예시
from sklearn.linear_model import LinearRegression
bias2, var = bias_variance_decomposition(LinearRegression(), X, y)
print(f"Bias^2: {bias2:.4f}, Variance: {var:.4f}")
```
- 여러 번 데이터를 샘플링(bootstrap)하여 모델을 반복 학습, 예측값의 평균과 분산을 계산
- 실제 데이터의 노이즈는 측정이 어렵기 때문에, 보통 bias^2와 variance를 중점적으로 분석

---



##### 시계열 예측에서의 오차 분해

시계열 예측에서는 **예측오차 분산분해(Forecast Error Variance Decomposition, FEVD)**라는 방법을 사용하기도 함

- 여러 시계열 변수가 있을 때, 미래 예측 오차의 분산이 각 변수에 의해 얼마나 설명되는지 계산
- 예측 시점마다 오차의 분산을 구하고, 각 변수의 기여율을 %로 산출하여, 어떤 요인이 예측에 가장 큰 영향을 주는지 파악

---

### 4. 실무 적용 팁

- **모델 복잡도를 변화시키며(예: 다항식 차수, 트리 깊이 등) 오차 분해를 반복**하면, 어느 구간에서 bias와 variance가 역전되는지 확인할 수 있음
- **학습 곡선과 함께 사용**하면, 모델의 개선 방향(더 복잡하게/더 단순하게/데이터 추가 필요 등)을 명확히 알 수 있음

---

##### 구성 요소 비교
| 요소 | 특징 | 대응 전략 |
|------|------|----------|
| **편향** | 모델의 과도한 단순화 | 계층 추가, 특징 공간 확장 |
| **분산** | 훈련 데이터 과민 반응 | L2 정규화, 앙상블 학습 |
| **노이즈** | 데이터 내재적 오차 | 데이터 품질 개선 |

#### 통합 접근법
1. **학습 곡선으로 초기 진단**  
   - 검증 손실이 15% 이상 상승 시 과적합 의심
2. **오차 분해로 근본 원인 규명**  
   - 편향 > 분산: 모델 용량 증가  
   - 분산 > 편향: 드롭아웃 비율 0.3~0.6으로 조정
3. **하이퍼파라미터 최적화**  
   - BayesianOptimization으로 50% 빠른 수렴 달성
